## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [2]:
from dataclasses import dataclass, field
from enum import Enum
from typing import List

class BiteLevel(Enum):
    Beginner = "Beginner"
    Intermediate = "Intermediate"
    Advanced = "Advanced"

@dataclass(order=True)
class Bite:
    number: int
    title: str
    level: BiteLevel = field(default=BiteLevel.Beginner, compare=False)
    
    def __str__(self):
        return f"{self.number} - {self.title} ({self.level.value})"

bites: List[Bite] = []
bites.append(Bite(154, 'Escreva uma dataclass', BiteLevel.Intermediate))
bites.append(Bite(1, 'Some n valores'))  # level padrão = Beginner
bites.append(Bite(37, 'Reescreva um loop com recursão', BiteLevel.Intermediate))

bites.sort()

for b in bites:
    print(b)


1 - Some n valores (Beginner)
37 - Reescreva um loop com recursão (Intermediate)
154 - Escreva uma dataclass (Intermediate)


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

metadata = MetaData()
metadata.reflect(engine)

Base = automap_base(metadata=metadata)
Base.prepare()

session = Session(engine)

Tracks = Base.classes.tracks
Albums = Base.classes.albums
Artists = Base.classes.artists
InvoiceItems = Base.classes.invoice_items


NameError: name 'metadata' is not defined

In [ ]:
### Imprima os três primeiros registros da tabela tracks
res_tracks = session.scalars(select(Tracks).limit(3))
print("3 primeiros registros de tracks:")
for t in res_tracks:
    print(t.TrackId, t.Name)
print('-'*50)

In [ ]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
stmt = select(Tracks.Name, Albums.Title).join(Albums, Tracks.AlbumId == Albums.AlbumId).limit(20)
res = session.execute(stmt)
print("Nome da faixa e título do álbum (primeiras 20 faixas):")
for name, title in res:
    print(name, "-", title)
print('-'*50)

In [ ]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
res_invoice = session.scalars(select(InvoiceItems).limit(10))
print("10 primeiras vendas de faixas:")
for i in res_invoice:
    print(f"InvoiceItemId: {i.InvoiceLineId}, TrackId: {i.TrackId}, Quantity: {i.Quantity}, UnitPrice: {i.UnitPrice}")
print('-'*50)


In [ ]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
stmt = select(Tracks.Name, InvoiceItems.Quantity).join(Tracks, Tracks.TrackId == InvoiceItems.TrackId).limit(10)
res = session.execute(stmt)
print("Nomes das faixas e quantidade vendida (10 primeiras vendas):")
for name, qty in res:
    print(name, "-", qty)
print('-'*50)

In [ ]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
stmt = select(
    Artists.Name,
    func.sum(InvoiceItems.Quantity).label("total_vendas")
).join(Albums, Albums.ArtistId == Artists.ArtistId)\
 .join(Tracks, Tracks.AlbumId == Albums.AlbumId)\
 .join(InvoiceItems, InvoiceItems.TrackId == Tracks.TrackId)\
 .group_by(Artists.Name)\
 .order_by(desc("total_vendas"))\
 .limit(10)

res = session.execute(stmt)
print("10 artistas que mais venderam:")
for artist, total in res:
    print(artist, "-", total)
